In [15]:
import sqlite3
from sqlalchemy import create_engine
import pandas as pd
from pandas.io import sql
from os import listdir
import datetime
import numpy as np

## ingestion

In [7]:
hist_dir = "../earnings_calendar_history/"
hist_files = [l for l in listdir(hist_dir) if l.endswith('.csv')]
# hist_files

In [68]:
database = "../main.db"

In [106]:
def convert_hist_file(hist_file: str) -> pd.DataFrame:
    col_mapper = {
            "TICKER\nNO MATCHES":"ticker",
            "MKT CAP":"cap",
            "EPS ESTIMATE":"eps_estimate",
            "REPORTED EPS":"eps_reported",
            "SURPRISE":"surprise",
            "SURPRISE %":"surprise_pct",
            "REVENUE FORECAST":"revenue_fx",
            "REVENUE ACTUAL":"revenue",
            "DATE":"eps_date",
            "PERIOD ENDING":"period_ending"
    }
    tb = pd.read_csv(hist_dir+hist_file)\
        .rename(columns=col_mapper)

    for col in col_mapper.values():
        if col not in tb.columns:
            tb[col] = np.nan

    tb = tb[col_mapper.values()]

    tb['file_date'] = hist_file.replace('.csv','')

    tb.replace('—',np.nan, inplace=True)
    
    def multiplier(s:str):
        s = str(s)
        if 'K' in s:
            return 1000.0
        if 'M' in s:
            return 1000_000.0
        if 'B' in s:
            return 1000_000_000.0
        if 'T' in s:
            return 1000_000_000_000.0
        return 1.0

    convert_numerical = lambda s:\
        float( str(s).replace('%','')\
                .replace('USD','')\
                .replace('−','-')\
                .replace('T','')\
                .replace('M','')\
                .replace('B','')\
                .replace('K','') )\
        *multiplier(s)

    for col in [
        "cap",
        "eps_estimate",
        "eps_reported",
        "surprise",
        "surprise_pct",
        "revenue_fx",
        "revenue"
    ]:
        tb[col]=tb[col].apply(convert_numerical)

    tb['name']   = tb['ticker'].apply(lambda s: s.split(';')[1])
    tb['ticker'] = tb['ticker'].apply(lambda s: s.split(';')[0])

    for col in ['eps_date','period_ending','file_date']:
        tb[col] = pd.to_datetime(tb[col])
    
    return tb

convert_hist_file(hist_files[28]).head(5)

,ticker,cap,eps_estimate,eps_reported,surprise,surprise_pct,revenue_fx,revenue,eps_date,period_ending,file_date,name
0,AAPL,2.478000e+12,1.27,1.29,0.02,1.74,8.873900e+10,NaN,NaT,NaT,2022-10-29,APPLE INC
1,MSFT,1.758000e+12,2.31,2.35,0.04,1.66,4.966300e+10,NaN,NaT,NaT,2022-10-29,MICROSOFT CORP.
2,GOOG,1.248000e+12,1.26,1.06,-0.20,-16.10,7.100100e+10,NaN,NaT,NaT,2022-10-29,ALPHABET INC (GOOGLE) CLASS C
3,AMZN,1.055000e+12,0.22,0.28,0.06,28.41,1.273870e+11,NaN,NaT,NaT,2022-10-29,"AMAZON.COM, INC."
4,XOM,4.613570e+11,3.86,4.45,0.59,15.22,1.045940e+11,NaN,NaT,NaT,2022-10-29,EXXON MOBIL CORPORATION


In [41]:
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Exception as e:
        print(e)

In [151]:
conn = sqlite3.connect(database)

In [109]:
sql_create_earnings_table = """ CREATE TABLE earnings (
                                        id integer PRIMARY KEY,
                                        ticker text NOT NULL,
                                        name text NOT NULL,
                                        cap decimal,
                                        eps_estimate decimal,
                                        eps_reported decimal,
                                        surprise decimal,
                                        surprise_pct decimal,
                                        revenue decimal,
                                        revenue_fx decimal,
                                        eps_date date,
                                        period_ending date,
                                        file_date date
                                    ); """
create_table(conn, sql_create_earnings_table)

run ```sqlite3 main.db```

In [110]:
for hist_file in hist_files[:]:
    convert_hist_file(hist_file).to_sql('earnings', conn, if_exists='append', index=False)

### checks...

In [111]:
pd.read_sql_query('SELECT * FROM earnings', conn)

,id,ticker,name,cap,eps_estimate,eps_reported,surprise,surprise_pct,revenue,revenue_fx,eps_date,period_ending,file_date
0,1,STZ,"CONSTELLATION BRANDS, INC.",4.695200e+10,2.09,2.37,-0.22,-10.39,2.103000e+09,2.017000e+09,2022-04-07 00:00:00,2022-02-28 00:00:00,2022-04-10 00:00:00
1,2,CAG,"CONAGRA BRANDS, INC.",1.655000e+10,0.58,0.58,-0.13,-21.67,2.914000e+09,2.846000e+09,2022-04-07 00:00:00,2022-02-28 00:00:00,2022-04-10 00:00:00
2,3,RPM,RPM INTERNATIONAL INC.,1.109900e+10,0.30,0.38,-0.05,-16.76,1.434000e+09,1.400000e+09,2022-04-06 00:00:00,2022-02-28 00:00:00,2022-04-10 00:00:00
3,4,LW,"LAMB WESTON HOLDINGS, INC.",9.631000e+09,0.44,0.73,0.29,65.25,9.550000e+08,9.685170e+08,2022-04-07 00:00:00,2022-02-28 00:00:00,2022-04-10 00:00:00
4,5,LEVI,LEVI STRAUSS & CO,7.488000e+09,0.42,0.46,0.06,15.19,1.592000e+09,1.547000e+09,2022-04-06 00:00:00,2022-02-28 00:00:00,2022-04-10 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4011,4012,ETCC,ENVIRONMENTAL TECTONICS CORP.,2.399000e+06,NaN,NaN,NaN,NaN,NaN,NaN,None,None,2023-01-14 00:00:00
4012,4013,SMIT,"SCHMITT INDUSTRIES, INC.",1.624000e+06,NaN,NaN,NaN,NaN,NaN,NaN,None,None,2023-01-14 00:00:00
4013,4014,WGNR,WEGENER CORP.,2.524680e+05,NaN,NaN,NaN,NaN,NaN,NaN,None,None,2023-01-14 00:00:00
4014,4015,PRXIQ,PREMIER EXHIBITIONS INC,7.930000e+02,NaN,NaN,NaN,NaN,NaN,NaN,None,None,2023-01-14 00:00:00


In [114]:
pd.read_sql_query('SELECT count(distinct ticker) as "unique symbols" FROM earnings', conn)

,unique symbols
0,1996


In [118]:
pd.read_sql_query("""
    SELECT sum(case when surprise>0 then 1.0 else 0.0 end)/count(1) as "positive EPS ratio" 
    FROM earnings""", conn)

,positive EPS ratio
0,0.487301


## load tickers history

In [152]:
sql_create_history_table = """ CREATE TABLE history (
                                        id integer PRIMARY KEY,
                                        ticker text NOT NULL,
                                        trade_date date,
                                        open decimal,
                                        high decimal,
                                        low decimal,
                                        close decimal,
                                        volume integer
                                    ); """
create_table(conn, sql_create_history_table)

In [121]:
pos_tickers = pd.read_sql_query('SELECT distinct ticker as "symbols" FROM earnings where surprise>0', conn)['symbols'].to_list()
len(pos_tickers)

1176

In [124]:
# !pip install yfinance

In [125]:
import yfinance as yf

In [127]:
start_date = '2022-01-01'
end_date   = datetime.datetime.now().strftime('%Y-%m-%d')
print(f"Getting tickers from {start_date} till {end_date}")

Getting tickers from 2022-01-01 till 2023-01-15


In [160]:
for i,ticker in enumerate(pos_tickers):
    print(f"{i}/{len(pos_tickers)} {ticker}...")
    try:
        tmp=yf.download(ticker, start_date, end_date)
    except Exception as e:
        print("Exception: "+str(e))
        continue
    
    tmp = tmp\
        .reset_index()\
        .drop(columns='Adj Close')\
        .rename(columns={
            "Date":"trade_date",
            "Open":"open",
            "Close":"close",
            "High":"high",
            "Low":"low",
            "Volume":"volume"
        })
    tmp['ticker'] = ticker
    
    tmp.to_sql('history', conn, if_exists='append', index=False)

0/1176 LW...
[*********************100%***********************]  1 of 1 completed
1/1176 LEVI...
[*********************100%***********************]  1 of 1 completed
2/1176 AYI...
[*********************100%***********************]  1 of 1 completed
3/1176 TRNO...
[*********************100%***********************]  1 of 1 completed
4/1176 TLRY...
[*********************100%***********************]  1 of 1 completed
5/1176 WDFC...
[*********************100%***********************]  1 of 1 completed
6/1176 SST...
[*********************100%***********************]  1 of 1 completed
7/1176 SCHN...
[*********************100%***********************]  1 of 1 completed
8/1176 GBX...
[*********************100%***********************]  1 of 1 completed
9/1176 NRIX...
[*********************100%***********************]  1 of 1 completed
10/1176 RGP...
[*********************100%***********************]  1 of 1 completed
11/1176 KRUS...
[*********************100%***********************]  1 of 1 comple

In [162]:
pd.read_sql_query('SELECT count(distinct ticker) as "unique symbols", count(1) as total_count FROM history', conn)

,unique symbols,total_count
0,1156,299589


In [150]:
conn.close()